In [41]:
# 내장 Library
import numpy as np
import json

#Tensorflow backend?
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf

from PIL import Image
import requests
from torchvision.io import read_image
import torchvision.transforms as transforms

# SHAP library
import shap
tf.compat.v1.disable_eager_execution()

## 이미지 불러오기 (Load Image)

In [42]:
X,y = shap.datasets.imagenet50()
to_explain = X[[39,41]]

# load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)

## Load Pretrained Vision Transformer 

In [57]:
import torch
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
model.eval()

Using cache found in /home/js/.cache/torch/hub/facebookresearch_deit_main


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=F

In [44]:
model.layers[-1].blocks[-1].norm1

LayerNorm((768,), eps=1e-05, elementwise_affine=True)

In [45]:
X.shape

(50, 224, 224, 3)

In [52]:
X = X.reshape((50,3,224,224))
X = torch.from_numpy(X)

TypeError: expected np.ndarray (got Tensor)

In [55]:
to_explain=to_explain.reshape((2,3,224,224))

In [50]:
to_explain = torch.from_numpy(to_explain)

In [59]:
e = shap.GradientExplainer(model, X)
shap_values = e.shap_values(to_explain)

KeyboardInterrupt: 

In [60]:
# explain how the input to the 7th layer of the model explains the top two classes
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input_resolution], [tf.keras.applications.imagenet_utils.preprocess_input(x.copy(),mode='torch')]))
    return K.get_session().run(model.layers[-1].blocks[layer].norm1, feed_dict)

In [65]:
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [66]:
map2layer(X, -1)

RuntimeError: The Session graph is empty. Add operations to the graph before calling run().

In [59]:
e = shap.GradientExplainer(
    (model.layers[-1].blocks[-1].input_resolution, model.layers[-1].blocks[-1].norm1),
    map2layer(X, -1),
    local_smoothing=0 # std dev of smoothing noise
)

RuntimeError: The Session graph is empty. Add operations to the graph before calling run().

In [96]:
target_layers = [model.blocks[-1].norm1]

In [ ]:
# X=np.array(image)/255

In [98]:
# data preprocess
# x=tf.compat.v1.keras.applications.imagenet_utils.preprocess_input(X,mode='torch')

In [ ]:
# def map2layer(X,blocks):
#     feed_dict = dict(zip([model.blocks[0].norm1],[X]))
#     return K.get_session().run(model.blocks[blocks].norm1,feed_dict)

In [7]:
image.shape
import torchvision.transforms as T
resize = T.Resize((224,224))
image = resize(image)

In [8]:
x=image

In [10]:
e = shap.GradientExplainer(model,x)

In [17]:
shap_values,indexes=e.shap_values(x,ranked_outputs=1)

In [20]:
e.shap_values()

In [18]:
shap.image_plot(shap_values)

AttributeError: 'NoneType' object has no attribute 'shape'

In [114]:
shap_values,indexes = e.shap_values(map2layer(to_explain, 7), ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[str(x)][1])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)

NameError: name 'to_explain' is not defined

In [ ]:
# explain how the input to the 7th layer of the model explains the top two classes
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)
e = shap.GradientExplainer(
    (model.layers[7].input, model.layers[-1].output),
    map2layer(X, 7),
    local_smoothing=0 # std dev of smoothing noise
)